In [1]:
# Enable GPU memory growth to avoid OOM errors
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Verify GPU is available
print("GPUs available:", len(tf.config.list_physical_devices('GPU')))

GPUs available: 1


In [ ]:
# !pip install tensorflow
import tensorflow as tf
# Set GPU devices to be invisible to TensorFlow
tf.config.set_visible_devices(gpus, 'GPU')

In [6]:
import os
# import cv2
import math
# import pafy
import random
import numpy as np
import datetime as dt
import ast
import re
# from moviepy.editor import *

import matplotlib.pyplot as plt
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from keras.layers import *
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, LSTM, Bidirectional, TimeDistributed
from keras.applications.vgg16 import VGG16
from keras import regularizers
from sklearn.svm import SVC
import pickle
import gc
from tensorflow.keras import backend as K


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
seed_constant = 23
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)


root_dir = '/content/drive/MyDrive/25mai0020'
os.chdir('/content/drive/MyDrive/25mai0020')
contents = os.listdir(root_dir)
for item in contents:
  print(item)

dataset
kfold1.ipynb
FLIP (1).ipynb
subpath_features
base_eachfold.ipynb
35runfirst.ipynb
LRCN_model_structure_plot.png
each_fold (1).ipynb


In [9]:
plt.figure(figsize = (20, 20))
# all_classes_names = ['brush_hair','catch','chew','climb','climb_stairs','dive','draw_sword','dribble','drink','eat','fall_floor','fencing','flic_flac','golf','handstand','hit','hug','jump','kick','kick_ball','kiss','laugh','pick','pullup','punch','push','pushup','ride_bike','ride_horse','shake_hands','shoot_bow','shoot_gun','sit','situp','smile','smoke','stand','swing_baseball','sword','sword_exercise','talk','throw','turn','wave','cartwheel','pour','run','shoot_ball','somersault','clap']
all_classes_names = ['brush_hair','catch','chew','climb','climb_stairs']
# Specify the height and width to which each video frame will be resized in our dataset.
IMAGE_HEIGHT , IMAGE_WIDTH = 64, 64

# Specify the number of frames of a video that will be fed to the model as one sequence.
SEQUENCE_LENGTH = 30

# Specify the list containing the names of the classes used for training. Feel free to choose any set of classes.
CLASSES_LIST = all_classes_names

<Figure size 2000x2000 with 0 Axes>

In [10]:
import pandas as pd

if not os.path.exists("subpath_features"):
    os.makedirs("subpath_features")


csv_path2 = os.path.join("subpath_features", "lables.npy")
csv_path3 = os.path.join("subpath_features", "features.npy")

In [11]:
CLASSES_LIST = CLASSES_LIST[0:5]

labels = np.load(csv_path2)

one_hot_encoded_labels = to_categorical(labels)

print(one_hot_encoded_labels.shape)
print(one_hot_encoded_labels)

labels.shape

(834, 5)
[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]


(834,)

In [12]:
def plot_metric(model_training_history, metric_name_1, metric_name_2, plot_name):
    '''
    This function will plot the metrics passed to it in a graph.
    Args:
        model_training_history: A history object containing a record of training and validation
                                loss values and metrics values at successive epochs
        metric_name_1:          The name of the first metric that needs to be plotted in the graph.
        metric_name_2:          The name of the second metric that needs to be plotted in the graph.
        plot_name:              The title of the graph.
    '''

    # Get metric values using metric names as identifiers.
    metric_value_1 = model_training_history.history[metric_name_1]
    metric_value_2 = model_training_history.history[metric_name_2]

    # Construct a range object which will be used as x-axis (horizontal plane) of the graph.
    epochs = range(len(metric_value_1))

    # Plot the Graph.
    plt.plot(epochs, metric_value_1, 'blue', label = metric_name_1)
    plt.plot(epochs, metric_value_2, 'red', label = metric_name_2)

    # Add title to the plot.
    plt.title(str(plot_name))

    # Add legend to the plot.
    plt.legend()

In [17]:
def create_cnn_bilstm_model():
    # Define the input
    inputs = Input(shape=(SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3))

    # Load VGG16 with pretrained weights and exclude top layers
    cnn_base = VGG16(weights='imagenet', include_top=False, input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3))

    # Freeze CNN layers
    for layer in cnn_base.layers:
        layer.trainable = False

    # Apply CNN to each frame
    x = TimeDistributed(cnn_base)(inputs)
    x = TimeDistributed(Flatten())(x)

    # BiLSTM layers
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    x = Dropout(0.5)(x)
    x = Bidirectional(LSTM(32))(x)

    # Output layer
    outputs = Dense(len(CLASSES_LIST), activation='softmax')(x)

    # Create the model
    model = Model(inputs, outputs)
    model.summary()

    return model

# Compile the model
check = 0

In [16]:

def create_cnn_bilstm_model1():
  cnn = VGG16(weights='imagenet', include_top=False, input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3))
  # Freeze the CNN layers
  for layer in cnn.layers:
    layer.trainable = False
  # Define the model architecture
  model = Sequential()
  model.add(TimeDistributed(cnn, input_shape = (SEQUENCE_LENGTH,IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
  model.add(TimeDistributed(Flatten()))
  model.add(Bidirectional(LSTM(64, return_sequences=True)))
  model.add(Dropout(0.5))
  model.add(Bidirectional(LSTM(32)))
  model.add(Dense(len(CLASSES_LIST), activation='softmax'))
  model.summary()

    # Return the constructed convlstm model.
  return model


# Compile the model
check = 0

In [18]:
model = create_cnn_bilstm_model()
check = 0

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 30, 64, 64, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 30, 2, 2, 512)  │    14,714,688 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 30, 2048)       │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 30, 128)        │     1,081,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 30, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 64)             │        41,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,838,085 (60.42 MB)

 Trainable params: 1,123,397 (4.29 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [ ]:
!pip install pydot
import pydot

In [ ]:
plot_model(model, to_file = 'LRCN_model_structure_plot.png', show_shapes = True, show_layer_names = True, show_layer_activations=True)

In [19]:
npypath = os.path.join("subpath_features",  f"features.npy")

k = 3
num_epochs = 10
batch_size = 32
histories=[]
scores = []


In [20]:
def loads() :
    features = np.load(npypath)
    print(features.shape)
    # Create KFold object
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    x = kf.split(features)
    splits = list(x)
    print(len(splits))
    with open('splits.pkl', 'wb') as f:
        pickle.dump(splits, f)
loads()

(834, 30, 64, 64, 3)
3


In [21]:
try:
    with open('scores.pkl', "rb") as file:
        scores = pickle.load(file)
        print("Exist")
except FileNotFoundError:
    scores = {}
    with open('scores.pkl', 'wb') as b:
        pickle.dump(scores, b)
    print("new dict")


new dict


In [22]:
print(CLASSES_LIST)

['brush_hair', 'catch', 'chew', 'climb', 'climb_stairs']


In [23]:
splits = []
with open('splits.pkl', 'rb') as f:
    splits = pickle.load(f)
len(splits[0][0])

556

**CLASSIFICATION REPORT**


In [24]:
#Function to compute classification report
def compute_classification_report(model, y_true , X_test): # y_true = correct , X_test = predicted
    y_pred = model.predict(X_test)

    report = classification_report(y_true, y_pred, target_names = CLASSES_LIST)
    print("report : " , report)

    return report

**PLOTTING**

In [25]:
def plot_graph(training, validation, label_train, label_val, title):
    # Create a range of epochs (x-axis)
    epochs = range(1, len(training) + 1)

    # Plot the training and validation values
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, training, label=label_train, marker='o', linestyle='-')
    plt.plot(epochs, validation, label=label_val, marker='o', linestyle='--')

    # Label the axes and add a legend
    plt.xlabel('Epochs')
    plt.ylabel('Value')
    plt.title(title)
    plt.legend()

    # Display the plot
    plt.grid(True)
    plt.show()


In [19]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5456767868690409450
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14619377664
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4762338511507512988
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [ ]:
tf.debugging.set_log_device_placement(True)

# Create some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [26]:
bidirectional_layers = [layer for layer in model.layers if 'bidirectional' in layer.name]
for i in bidirectional_layers:
  print(i)

<Bidirectional name=bidirectional, built=True>
<Bidirectional name=bidirectional_1, built=True>


In [27]:
# --- load splits & scores (existing behavior) ---
with open('splits.pkl', 'rb') as f:
    splits = pickle.load(f)

# load existing scores dict (if file exists). Overwrite approach avoids multiple appended dicts.
scores = {}
try:
    with open('scores.pkl', 'rb') as f:
        scores = pickle.load(f)
except Exception:
    scores = {}

# Containers
intermediate_model = None
histories = []                      # make sure declared earlier if used elsewhere
classification_reports = []
training_accuracy_values = []
validation_accuracy_values = []
training_loss_values = []
validation_loss_values = []

# fold selection
kfold_value = 1
endKfold_value = kfold_value + 1

model_num = check  # check must be defined earlier (your code used this)
im = kfold_value

# consistent batch_size
batch_size = 8   # used during fit; keep same for predict if you like

for j, (train_indices, val_indices) in enumerate(splits[kfold_value:endKfold_value]):
    if im in scores:
        print("im :", im, "already computed, skipping.")
        im += 1
        continue

    # load features
    features = np.load(npypath)
    features = np.asarray(features)

    X_train, y_train = features[train_indices], one_hot_encoded_labels[train_indices]
    X_val, y_val = features[val_indices], one_hot_encoded_labels[val_indices]

    # Create a fresh model instance for this fold
    with tf.device('/GPU:0'):
        model = create_cnn_bilstm_model()                 # MAKE SURE this returns a keras.Model/Sequential
        # Ensure model graph is built (so model.input is defined)
        model.build(input_shape=(None, SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3))
        early_stopping_callback = EarlyStopping(monitor='val_loss', patience=15, mode='min', restore_best_weights=True)
        model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

        # bookkeeping for name-based selection (if you still need model_num)
        model_num = check
        check = check + 1

        # Train
        LRCN_model_training_history = model.fit(
            x=X_train, y=y_train,
            epochs=3,
            batch_size=batch_size,
            validation_data=(X_val, y_val),
            callbacks=[early_stopping_callback]
        )

    # save history values for plotting / record
    training_accuracy_values = LRCN_model_training_history.history.get('accuracy', [])
    validation_accuracy_values = LRCN_model_training_history.history.get('val_accuracy', [])
    training_loss_values = LRCN_model_training_history.history.get('loss', [])
    validation_loss_values = LRCN_model_training_history.history.get('val_loss', [])
    histories.append(LRCN_model_training_history)

    # Build intermediate model (extract last Bidirectional layer output)
    with tf.device('/GPU:0'):
        # ensure model tensors exist (build or a dummy pass - build above is usually enough)
        # But to be safe, do a dummy forward pass if session still complains:
        dummy_input = np.zeros((1, SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3))
        try:
            _ = model(dummy_input)
        except Exception:
            pass

        bidirectional_layers = [layer for layer in model.layers if 'bidirectional' in layer.name]
        if not bidirectional_layers:
            raise RuntimeError("No bidirectional layers found in model.layers; check create_cnn_bilstm_model()")
        # choose last bidirectional by default
        layer_output = bidirectional_layers[-1].output
        intermediate_model = tf.keras.models.Model(inputs=model.input, outputs=layer_output)

    # Create train/test split for SVM features (this is your existing behaviour)
    fat_train, fat_test, labels_train, labels_test = train_test_split(
        features, one_hot_encoded_labels, test_size=0.25, shuffle=True, random_state=seed_constant
    )

    # Predict intermediate features (use GPU)
    with tf.device('/GPU:0'):
        X_train_features = intermediate_model.predict(fat_train, batch_size=batch_size, verbose=1)
        X_test_features  = intermediate_model.predict(fat_test, batch_size=batch_size, verbose=1)

    # reshape features for SVM
    X_train_svm = X_train_features.reshape(X_train_features.shape[0], -1)
    X_test_svm  = X_test_features.reshape(X_test_features.shape[0], -1)
    print("xtrain shape ", X_train_svm.shape)
    print("xtest shape ", X_test_svm.shape)

    labels_train_1d = np.argmax(labels_train, axis=1)
    labels_test_1d = np.argmax(labels_test, axis=1)

    # Train and evaluate SVM
    svm = SVC(kernel='linear', probability=True, C=1.0)
    svm.fit(X_train_svm, labels_train_1d)
    accuracy = svm.score(X_test_svm, labels_test_1d)
    pred = svm.predict(X_test_svm)
    print("Accuracy:", accuracy)

    # store score safely (overwrite scores file each fold)
    scores[im] = accuracy
    with open('scores.pkl', 'wb') as sf:
        pickle.dump(scores, sf)

    # classification report and commit (you call compute_classification_report)
    report = compute_classification_report(svm, labels_test_1d, X_test_svm)
    classification_reports.append(report)
    with open('average_classification_report.pkl', 'wb') as f:
        pickle.dump(classification_reports, f)

    # Save artifacts for this fold (overwrite previous ones for the same fold)
    model.save(f'model_fold_{im+1}.h5')
    intermediate_model.save(f'intermediate_model_fold_{im+1}.h5')
    pickle.dump(svm, open(f'svm_{im+1}.pickle', "wb"))
    pickle.dump(histories, open(f'histories{im+1}.pickle', "wb"))
    pickle.dump(pred, open(f'pred{im+1}.pickle', "wb"))
    # if you want to store SVM object also as z
    pickle.dump(svm, open(f'z{im+1}.pickle', "wb"))

    # Clean up Keras session to prevent name/graph buildup in subsequent folds
    K.clear_session()
    del model
    del intermediate_model
    gc.collect()

    im += 1

    # plotting
    plot_graph(training_accuracy_values, validation_accuracy_values, 'Training Accuracy', 'Validation Accuracy', 'Accuracy vs. Epochs')
    plot_graph(training_loss_values, validation_loss_values, 'Training Loss', 'Validation Loss', 'Loss vs. Epochs')


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 30, 64, 64, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 30, 2, 2, 512)  │    14,714,688 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 30, 2048)       │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 30, 128)        │     1,081,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 30, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ (None, 64)             │        41,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,838,085 (60.42 MB)

 Trainable params: 1,123,397 (4.29 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

Epoch 1/3
70/70 ━━━━━━━━━━━━━━━━━━━━ 50s 416ms/step - accuracy: 0.4801 - loss: 1.3164 - val_accuracy: 0.7122 - val_loss: 0.7696
Epoch 2/3
70/70 ━━━━━━━━━━━━━━━━━━━━ 23s 323ms/step - accuracy: 0.7916 - loss: 0.6081 - val_accuracy: 0.7986 - val_loss: 0.5616
Epoch 3/3
70/70 ━━━━━━━━━━━━━━━━━━━━ 22s 322ms/step - accuracy: 0.8882 - loss: 0.3516 - val_accuracy: 0.8345 - val_loss: 0.6127
79/79 ━━━━━━━━━━━━━━━━━━━━ 25s 255ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 6s 204ms/step
xtrain shape  (625, 64)
xtest shape  (209, 64)


Accuracy: 0.937799043062201
report :                precision    recall  f1-score   support

  brush_hair       0.90      0.92      0.91        48
       catch       0.97      1.00      0.99        36
        chew       0.95      0.93      0.94        44
       climb       0.92      0.90      0.91        39
climb_stairs       0.95      0.95      0.95        42

    accuracy                           0.94       209
   macro avg       0.94      0.94      0.94       209
weighted avg       0.94      0.94      0.94       209



NameError: name 'K' is not defined

In [28]:
scores = {}
with open('scores.pkl', 'rb') as f:
    try:
        while True:
            k = pickle.load(f)
            scores.update(k)
    except EOFError:
        pass
scores

{1: 0.937799043062201}

In [ ]:
scores = {0: 0.9385474860335196, 1: 0.9925650557620818}
with open('scores.pkl', 'wb') as f:
    scores = pickle.dump(scores , f)
scores

In [ ]:
output_directory = 'Youtube_Videos'
os.makedirs(output_directory, exist_ok = True)